# Simple ADK Agents

In [ ]:
! pip install google-cloud-aiplatform[adk] --quiet --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# @title Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}
STAGING_BUCKET = "gs://agent-engine-staging-bucket-anita" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "qwiklabs-gcp-02-8501559ee091" # @param {type:"string"}

print(PROJECT_ID)

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

qwiklabs-gcp-02-8501559ee091


## Simple Weather Agent

NWS Weather Function

In [ ]:
import requests
from typing import Dict, Any, Optional, List

def get_weather_forecast(latitude: float, longitude: float) -> Optional[List[Dict[str, Any]]]:
    """
    Retrieves the weather forecast from the National Weather Service (NWS) API.

    This function takes a latitude and longitude, first determines the
    correct NWS forecast grid endpoint, and then fetches the weather
    forecast for that location.

    Args:
        latitude: The latitude of the location.
        longitude: The longitude of the location.

    Returns:
        A list of dictionaries, where each dictionary represents a forecast
        period (e.g., 'Tonight', 'Thursday'). Returns None if the forecast
        cannot be retrieved.
    """
    # NWS API requires a User-Agent header such as app name.
    headers = {
        'User-Agent': 'my-weather-app-av'
    }

    # --- Step 1: Get the forecast endpoint URL from the points endpoint ---
    points_url = f"https://api.weather.gov/points/{latitude},{longitude}"

    try:
        points_response = requests.get(points_url, headers=headers, timeout=10)
        # Raise an exception for bad status codes (4xx or 5xx)
        points_response.raise_for_status()
        points_data = points_response.json()

        # Extract the forecast URL from the response
        forecast_url = points_data.get('properties', {}).get('forecast')

        if not forecast_url:
            print("Error: Could not find forecast URL in the API response.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error connecting to NWS points API: {e}")
        return None
    except ValueError: # Catches JSON decoding errors
        print("Error: Could not decode JSON response from points API.")
        return None

    # --- Step 2: Get the actual forecast from the forecast URL ---
    try:
        forecast_response = requests.get(forecast_url, headers=headers, timeout=10)
        forecast_response.raise_for_status()
        forecast_data = forecast_response.json()

        # The detailed forecast is in the 'periods' list
        return forecast_data.get('properties', {}).get('periods')

    except requests.exceptions.RequestException as e:
        print(f"Error connecting to NWS forecast API: {e}")
        return None
    except ValueError:
        print("Error: Could not decode JSON response from forecast API.")
        return None

Get Lat/Long Function

In [ ]:
def get_lat_lon_from_place(api_key: str, address: str):
    """
    Converts a physical address or city name to latitude and longitude using
    the Google Maps Geocoding API.

    Args:
        api_key: Your valid Google Maps Geocoding API key.
        address: The city, street address, or place name to geocode.

    Returns:
        A tuple containing the latitude and longitude as floats (lat, lon).
        Returns None if the address cannot be geocoded or if an API
        error occurs.
    """
    # API endpoint URL
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    # Parameters for the API request
    params: Dict[str, str] = {
        'address': address,
        'key': api_key,
    }

    try:
        # Make the GET request to the Google Maps API
        response = requests.get(base_url, params=params, timeout=10)
        # Raise an HTTPError for bad responses (4xx or 5xx)
        response.raise_for_status()
        # Parse the JSON response
        data: Dict[str, Any] = response.json()

        # Check the status of the API response
        if data.get('status') == 'OK' and data.get('results'):
            # Extract the geometry and location data
            location = data['results'][0]['geometry']['location']
            latitude: float = location['lat']
            longitude: float = location['lng']
            return (latitude, longitude)
        else:
            # Print an error message if geocoding failed
            error_message = data.get('error_message', 'No results found.')
            print(f"Geocoding Error: {data.get('status')} - {error_message}")
            return None

    except requests.exceptions.RequestException as e:
        # Handle connection errors, timeouts, etc.
        print(f"Error connecting to Google Maps API: {e}")
        return None
    except (ValueError, KeyError, IndexError):
        # Handle errors from malformed JSON or unexpected structure
        print("Error: Could not parse response from Google Maps API.")
        return None

Get weather for city

In [ ]:
def get_weather_city(city: str) -> dict:
    """
    Retrieves the weather forecast for the city

    This function prompts for the input city and
    prints the report in a user friendly format

    Args:
        city: The city or place name to geocode.

    Returns:
        A list of weather parameters such as temperature, wind etc
        as returned by NWS API
    """


    # IMPORTANT: Replace "YOUR_API_KEY" with your actual Google Maps API key.
    gmaps_api_key = "REVOKED KEY"  # @param {type:"string"}

    if gmaps_api_key == "":
        print("To run this , you must replace 'YOUR_API_KEY' in the")
        print("script with your actual Google Maps Geocoding API key.")

    else:
        # Get city from user input
        #city = input("Enter a city name to get the weather forecast: ")

        # Get coordinates for the city
        coordinates = get_lat_lon_from_place(gmaps_api_key, city)

        if coordinates:
            print(f"\nCoordinates for {city}: Latitude={coordinates[0]}, Longitude={coordinates[1]}")

            # Get weather forecast using the coordinates
            forecast_periods = get_weather_forecast(coordinates[0], coordinates[1])

            if forecast_periods:
                first_period = forecast_periods[0]
                weather_text="\n--- Current Forecast ---\n" \
                "Time: ",first_period.get('name'),"\n" \
                "Temperature: " ,first_period.get('temperature'),"°",first_period.get('temperatureUnit'),"\n" \
                "Wind: ",first_period.get('windSpeed')," from the ",first_period.get('windDirection'),"\n" \
                "Forecast: ",first_period.get('detailedForecast'),"\n"
                return weather_text
            else:
                print("\nFailed to retrieve weather forecast for the location.")
        else:
            print(f"\nCould not find coordinates for '{city}'.")

In [ ]:
# @title Test the agent locally
from google.adk.agents import Agent
from google.adk.agents import LlmAgent, BaseAgent
from google.adk.tools import google_search

# from google.adk.models.lite_llm import LiteLlm

AGENT_MODEL = "gemini-2.0-flash" # Can be a string for Gemini or a LiteLlm object
weather_agent = LlmAgent(
    name="weather_agent_v1",
    model=AGENT_MODEL,
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather_city' tool to find the information. "
                ,
    tools=[get_weather_city], # Pass the function directly
)
google_search_agent = LlmAgent(

    name="search_agent_v1",
    model=AGENT_MODEL,
    description=(
        "You can search Google and return answers from the Internet."
    ),
    instruction=(
        """You are a helpful Chatbot.
        If you don't know something, then Search Google for the answer."""
    ),
    tools=[google_search]
)
main_agent = LlmAgent(
 name="main_agent",
 model=AGENT_MODEL,
 description="Provides Answers to Users Questions.",
 instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather_city' tool to find the information. "
                "Otherwise use google_search_agent to answer questions ",
 #tools=[agent_tool.AgentTool(agent=google_search_agent)],
 sub_agents=[weather_agent],
)

print(f"Agent '{weather_agent.name}' created using model '{AGENT_MODEL}'.")

Agent 'weather_agent_v1' created using model 'gemini-2.0-flash'.


In [ ]:
# @title Create an App that uses the agent

from vertexai.preview import reasoning_engines
app = reasoning_engines.AdkApp(
    agent=main_agent,
    enable_tracing=False,
)

In [ ]:

# @title Send a message to the locally running agent
# @title Create a user session
user_id = "test-user-id"
session = app.create_session(user_id=user_id)

print(session.id)
from IPython.display import Markdown, display

#test weather
for event in app.stream_query(
    user_id=user_id,
    session_id=session.id,
    message="weather in Austin",
):
    lastevent = event

display(Markdown(lastevent["content"]["parts"][0]["text"]))

#test google search
for event in app.stream_query(
    user_id=user_id,
    session_id=session.id,
    message="when was Google founded?",
):
    lastevent = event

display(Markdown(lastevent["content"]["parts"][0]["text"]))



08f3a712-7cce-4fae-b0c7-f001b01b1daa



Coordinates for Austin: Latitude=30.267153, Longitude=-97.7430608


The weather in Austin tonight will be partly cloudy, with a low around 75°F. The wind will be around 0 mph from the south.


Google was founded on September 4, 1998.
